In [8]:
import pandas as pd
import joblib
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error
import re

# Load the saved scaler
scaler = joblib.load('scaler.pkl')

# Load the saved model for N(Ef) prediction
final_model = joblib.load('GB_final_model.pkl')


# Initialize an empty DataFrame
alloy_df = pd.DataFrame()

# Input alloy composition
while True:
    composition = input("Enter refractory alloy and the % of elements.\n"
                        "For example: Cr25Mo15Nb20Ta20W20 (Total % = 25+15+20+20+20 = 100)\n"
                        "Note: Sum of % should be between 99% and 101%.\n")

    try:
        # Extract elements and percentages using regex
        matches = re.findall(r'([A-Z][a-z]*)(\d*\.?\d+|\d+)', composition)
        elements = [element for element, _ in matches]
        percentages = [float(percentage) for _, percentage in matches]

        # Calculate total percentage
        total_percentage = sum(percentages)

        # Check if total percentage is within the allowed range [99%, 101%]
        if not (99 <= total_percentage <= 101):
            raise ValueError(
                f"The total atomic percentage for {composition} is {total_percentage}%, "
                "which is outside the allowed range [99%, 101%]. Please check your input."
            )

        # Populate the DataFrame
        alloy_df['Alloy'] = [composition]

        # Break the loop if the input is valid
        break

    except ValueError as ve:
        print(f"Error: {ve}")
    except Exception as e:
        print(f"Unexpected error: {e}")

# import feature functions
%run ML_functions.ipynb

# Calculate properties
try:
    # Mixing enthalpy
    alloy_df['Mixing Enthalpy'] = alloy_df['Alloy'].apply(lambda x: calculate_mixing_enthalpy(x, AB_mix_enthalpy))
    # VEC
    alloy_df['VEC'] = alloy_df['Alloy'].apply(lambda x: calculate_vec(x, pvec))
    # Melting temperature
    alloy_df['Melting Temperature'] = alloy_df['Alloy'].apply(lambda x: calculate_mt(x, mt))
    # Electronegativity
    alloy_df['Pauling Electronegativity'] = alloy_df['Alloy'].apply(lambda x: calculate_pauli(x, chi))
    # Entropy of mixing
    alloy_df['Entropy of Mixing'] = alloy_df['Alloy'].apply(calculate_entropy_of_mixing)

    # Add all 9 specified element columns
    all_elements = ['Cr', 'Mo', 'W', 'V', 'Nb', 'Ta', 'Ti', 'Zr', 'Hf']
    percentages_dict = {el: 0 for el in all_elements}  # Initialize all to 0

    # Update the dictionary with actual percentages from the alloy
    for el, percentage in matches:
        if el in percentages_dict:
            percentages_dict[el] = percentage

    # Add each element as a column in the DataFrame
    for el in all_elements:
        alloy_df[el] = percentages_dict.get(el, 0)

    # Make predictions
    X_new = alloy_df.drop(columns=['Alloy'])
    X_new_normalized = scaler.transform(X_new)
    X_new_normalized = pd.DataFrame(X_new_normalized, columns=scaler.feature_names_in_)
    pred = final_model.predict(X_new_normalized)[0]

    # Add predictions to the DataFrame
    alloy_df['N(Ef)'] = pred
    alloy_df['rec_DOSperAtom'] = pred

    print(f'DOS at Fermi of {composition} is: {pred:.2f}')

    # Display the final DataFrame
    # print("\nFinal DataFrame with Predictions:")
    # print(alloy_df)

except Exception as e:
    print(f"An error occurred during calculations: {e}")

Enter refractory alloy and the % of elements.
For example: Cr25Mo15Nb20Ta20W20 (Total % = 25+15+20+20+20 = 100)
Note: Sum of % should be between 99% and 101%.
 Mo10Zr30Hf20Ti20V20


Import Successful!
DOS at Fermi of Mo10Zr30Hf20Ti20V20 is: 1.42
